In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to the zip file in Google Drive
zip_file_path = '/content/drive/MyDrive/Content.zip'

# Check if the zip file exists
if os.path.exists(zip_file_path):
    print(f"The zip file '{zip_file_path}' exists.\n")

    # Define the directory where you want to extract the contents
    extracted_dir = '/content/Content/datasets'

    # Check if the extraction directory exists
    if os.path.exists(extracted_dir):
        print(f"The extraction directory '{extracted_dir}' exists.\n")
        # List directories and their contents
        print("Existing directories and their contents")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
The zip file '/content/drive/MyDrive/Content.zip' exists.

The extraction directory '/content/Content/datasets' exists.

Existing directories and their contents


In [3]:
from google.colab import drive
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to the zip file in Google Drive
zip_file_path = '/content/drive/MyDrive/Content.zip'

# Define the directory where you want to extract the contents
extracted_dir = '/content/Content/datasets'

# Check if the zip file exists
if os.path.exists(zip_file_path):
    print(f"The zip file '{zip_file_path}' exists.\n")

    # Check if the extraction directory exists
    if not os.path.exists(extracted_dir):
        print(f"The extraction directory '{extracted_dir}' does not exist.")
        print("Creating the directory...")
        os.makedirs(extracted_dir)
        print("Directory created successfully.\n")

    # Extract the zip file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_dir)
        print("Zip file extracted successfully.\n")

    # List the contents of the extraction directory
    # print("Contents of the extraction directory:")
    # for root, dirs, files in os.walk(extracted_dir):
    #     print(f"Directory: {root}")
    #     if files:
    #         print("Files:")
    #         for file in files:
    #             print(file)
    #     else:
    #         print("No files in this directory")
    #     print()
else:
    print(f"The zip file '{zip_file_path}' does not exist.")
    print("Please make sure the file exists and provide the correct path.")

# Define paths for the dataset
train_dir = os.path.join(extracted_dir, 'Content/datasets/train')
val_dir = os.path.join(extracted_dir, 'Content/datasets/val')
test_dir = os.path.join(extracted_dir, 'Content/datasets/test')

# Function to list directories and count images
def list_directories_and_count_images(path):
    for root, dirs, files in os.walk(path):
        print(f'Directory: {root}')
        if len(files) > 0:
            print(f'Contains {len(files)} files')

# Check the contents of each directory
print("Checking training directory:")
list_directories_and_count_images(train_dir)
print("\nChecking validation directory:")
list_directories_and_count_images(val_dir)
print("\nChecking test directory:")
list_directories_and_count_images(test_dir)

# Define ImageDataGenerators for loading and augmenting images
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    class_mode='categorical',  # Change to categorical
                                                    color_mode='grayscale')

val_generator = val_datagen.flow_from_directory(val_dir,
                                                target_size=(150, 150),
                                                batch_size=32,
                                                class_mode='categorical',  # Change to categorical
                                                color_mode='grayscale')

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(150, 150),
                                                  batch_size=32,
                                                  class_mode='categorical',  # Change to categorical
                                                  color_mode='grayscale')

# Print the number of samples in each generator
print(f'Number of training samples: {train_generator.samples}')
print(f'Number of validation samples: {val_generator.samples}')
print(f'Number of test samples: {test_generator.samples}')

# Ensure that we have sufficient images to proceed with training
if train_generator.samples > 0 and val_generator.samples > 0 and test_generator.samples > 0:
    # Define the CNN model
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(3, activation='softmax')  # Change to 3 units and softmax activation
    ])

    # Compile the model
    model.compile(loss='categorical_crossentropy',  # Change to categorical_crossentropy
                  optimizer='adam',
                  metrics=['accuracy'])

    # Define callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)
    model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss')

    # Train the model
    history = model.fit(train_generator,
                        steps_per_epoch=max(1, train_generator.samples // train_generator.batch_size),
                        epochs=50,
                        validation_data=val_generator,
                        validation_steps=max(1, val_generator.samples // val_generator.batch_size),
                        callbacks=[early_stopping, model_checkpoint])

    # Load the best model
    model.load_weights('best_model.h5')

    # Evaluate the model on the test set
    test_loss, test_acc = model.evaluate(test_generator, steps=max(1, test_generator.samples // test_generator.batch_size))
    print(f'Test accuracy: {test_acc:.2f}')

    # Plot training history
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend()
    plt.title('Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.legend()
    plt.title('Loss')

    plt.show()

    # Save the model
    model_path = '/content/drive/MyDrive/first_draft_classifier_v_13.h5'  # Replace with your desired path
    model.save(model_path)

else:
    print("Please ensure that your dataset directories contain images and try again.")


MessageError: Error: credential propagation was unsuccessful

In [ ]:
from google.colab import drive
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to the zip file in Google Drive
zip_file_path = '/content/drive/MyDrive/Content.zip'

# Define the directory where you want to extract the contents
extracted_dir = '/content/Content/datasets'

# Check if the zip file exists
if os.path.exists(zip_file_path):
    print(f"The zip file '{zip_file_path}' exists.\n")

    # Check if the extraction directory exists
    if not os.path.exists(extracted_dir):
        print(f"The extraction directory '{extracted_dir}' does not exist.")
        print("Creating the directory...")
        os.makedirs(extracted_dir)
        print("Directory created successfully.\n")

    # Extract the zip file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_dir)
        print("Zip file extracted successfully.\n")

    # List only directories in the extraction directory
    print("Directories after extraction:")
    for root, dirs, files in os.walk(extracted_dir):
        if dirs:
            for directory in dirs:
                print(os.path.join(root, directory))
else:
    print(f"The zip file '{zip_file_path}' does not exist.")
    print("Please make sure the file exists and provide the correct path.")

# Define paths for the dataset
train_dir = os.path.join(extracted_dir, 'Content/datasets/train')
val_dir = os.path.join(extracted_dir, 'Content/datasets/val')
test_dir = os.path.join(extracted_dir, 'Content/datasets/test')

# Define ImageDataGenerators for loading and augmenting images
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(150, 150),
                                                  batch_size=32,
                                                  class_mode='categorical',  # Change to categorical
                                                  color_mode='grayscale',
                                                  shuffle=False)  # Don't shuffle for easier comparison

# Load the trained model
model_path = '/content/drive/MyDrive/first_draft_classifier_v_12.h5'
model = tf.keras.models.load_model(model_path)

# Make predictions on the test set
predictions = model.predict(test_generator)

# Extract class labels from the generator
class_labels = list(test_generator.class_indices.keys())

# Get true labels
true_labels = test_generator.classes

# Get predicted labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate accuracy
accuracy = np.mean(true_labels == predicted_labels)
print(f'Test accuracy: {accuracy:.2f}')

# Plot some images with their predicted labels from multiple classes, limiting to 9 images in total
# plt.figure(figsize=(10, 10))
# class_labels = list(test_generator.class_indices.keys())
# num_images = min(9, len(test_generator))
# for i in range(num_images):
#     plt.subplot(3, 3, i + 1)
#     image, true_label_index = test_generator[i]
#     image = image[0]
#     predicted_label_index = np.argmax(model.predict(np.expand_dims(image, axis=0)))
#     true_label_index = true_label_index[0]  # Extracting the integer value
#     plt.imshow(image, cmap='gray')
#     plt.title(f'Predicted: {class_labels[predicted_label_index]}\nTrue: {class_labels[int(true_label_index[0])]}')
#     plt.axis('off')
# plt.tight_layout()
# plt.show()





Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
The zip file '/content/drive/MyDrive/Content.zip' exists.

Zip file extracted successfully.

Directories after extraction:
/content/Content/datasets/Content
/content/Content/datasets/__MACOSX
/content/Content/datasets/Content/datasets
/content/Content/datasets/Content/datasets/test
/content/Content/datasets/Content/datasets/train
/content/Content/datasets/Content/datasets/val
/content/Content/datasets/Content/datasets/test/TB
/content/Content/datasets/Content/datasets/test/Normal
/content/Content/datasets/Content/datasets/test/CPA
/content/Content/datasets/Content/datasets/train/TB
/content/Content/datasets/Content/datasets/train/Normal
/content/Content/datasets/Content/datasets/train/CPA
/content/Content/datasets/Content/datasets/val/TB
/content/Content/datasets/Content/datasets/val/Normal
/content/Content/datasets/Content/datasets/val/CPA
/content/Content/d

In [ ]:
from google.colab import drive, files
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Mount Google Drive
drive.mount('/content/drive')

# Function to preprocess an image
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(150, 150), color_mode="grayscale")
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)
    return np.expand_dims(img_array, axis=0)

# Upload files
uploaded = files.upload()

# Once the files are uploaded, you can access them like this:
for filename in uploaded.keys():
    print(f"Uploaded file: {filename}")
    # Preprocess the uploaded image
    img_path = f'/content/{filename}'
    img = preprocess_image(img_path)
    # Load the model
    model_path = '/content/drive/MyDrive/first_draft_classifier_v_12.h5'
    model = load_model(model_path)
    class_labels = train_generator.class_indices

    # Predict the class
    predictions = model.predict(img)
    # Get the predicted class index
    predicted_class_index = np.argmax(predictions)
    # Retrieve class names from the model
    # class_names = model.output_names[predicted_class_index]
    predicted_class_label = list(class_labels.keys())[predicted_class_index]
    print(f"Predicted index: {predicted_class_index}")
    print(f"Predicted class: {predicted_class_label}")
    # Display the image
    # plt.imshow(img.squeeze(), cmap='gray')  # Assuming grayscale image
    # plt.axis('off')
    # plt.show()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving Normal-590.png to Normal-590.png
Uploaded file: Normal-590.png
1/1 [==============================] - 0s 93ms/step
Predicted index: 0
Predicted class: CPA


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from google.colab import files
import cv2

def is_potential_chest_xray(image_path):
  """
  This function analyzes basic image properties to check for characteristics often found in chest X-rays.

  Args:
      image_path: Path to the image file.

  Returns:
      True if the image has properties suggestive of a chest X-ray, False otherwise.
  """
  # Load the image
  image = cv2.imread(image_path)

  # Check for grayscale image
  if len(image.shape) != 2 and image.shape[2] != 1:
    return False

  # Analyze intensity distribution (heuristic for chest X-ray range)
  num_pixels = image.shape[0] * image.shape[1]
  hist = cv2.calcHist([image], [0], None, [256], [0, 256])
  intensity_in_range = np.sum(hist[40:180])  # Adjusted range for chest X-rays
  if intensity_in_range / num_pixels < 0.6:
    return False

  # Look for ribcage pattern (very basic check)
  # Convert to grayscale and apply thresholding
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)[1]

  # Apply vertical edge detection
  edges = cv2.Canny(thresh, 50, 150, apertureSize=3)

  # Check for presence of vertical edges throughout the image (suggestive of ribs)
  vertical_edge_ratio = cv2.countNonZero(edges) / num_pixels
  if vertical_edge_ratio < 0.1:
    return False

  return True

# # Example usage
# image_path = "path/to/your/chest_xray.jpg"
# if is_potential_chest_xray(image_path):
#   print("Image has characteristics suggestive of a chest X-ray.")
# else:
#   print("Image is unlikely to be a chest X-ray.")


# Function to preprocess the uploaded image
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(150, 150), color_mode='grayscale')
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Load the trained model
model = load_model('best_model.h5')

# Define class labels
class_labels = ['Normal', 'TB', 'CPA']  # Assuming three classes: Normal, TB, CPA

def has_non_xray_colors(image_path, threshold=0.01):
    img = image.load_img(image_path)
    img_array = image.img_to_array(img)
    # Define typical pixel value ranges for X-ray images
    typical_range_min = 0
    typical_range_max = 100
    # Count the number of pixels falling outside the typical range
    non_xray_pixels = np.sum((img_array < typical_range_min) | (img_array > typical_range_max))
    # Calculate the proportion of non-X-ray pixels
    proportion_non_xray = non_xray_pixels / img_array.size
    # If the proportion exceeds the threshold, return True
    return proportion_non_xray > threshold



# Function to predict the class of the uploaded image with rejection option
def predict_image_class(image_path, threshold=0.5):
    if is_potential_chest_xray(image_path):
        print("Rejecting image with color")
        return None  # Reject the image if it has color
    img_array = preprocess_image(image_path)
    predictions = model.predict(img_array)[0]
    predicted_class = np.argmax(predictions)
    if predictions[predicted_class] < threshold:
        return "Uncertain - Image may not belong to any class"
    else:
        return class_labels[predicted_class]

# Upload the image file
uploaded = files.upload()

# Process and predict the class for each uploaded image
for filename in uploaded.keys():
    uploaded_image_path = filename
    predicted_class = predict_image_class(uploaded_image_path)
    print(f"Uploaded Image: {filename}, Predicted Class: {predicted_class}")


OSError: No file or directory found at /content/drive/MyDrive/first_draft_classifier_v_12.h5